In [ ]:
import h5py
import pylab as plt
from matplotlib.colors import LogNorm
import matplotlib.ticker as mtick
import numpy as np
import os
from scipy.stats import linregress
import seaborn as sns
import tensorflow as tf
import glob
from spacepy import pycdf
import warnings
import moms_fast
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import nnet_evaluate
import utils

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
hdf= h5py.File('/home/ubuntu/data/samples_train_n=50000_nosw.hdf')
phases = list(hdf.keys())

for phase in phases:
    print(phase.ljust(20), hdf[phase]['E'].shape[0])

# Figure 2

In [ ]:
files = [
    '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4A_dusk_flank.rfr001/moments_stats.hdf',
    '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4B_dayside.rfr001/moments_stats.hdf',
    '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4C_dawn_flank.rfr001/moments_stats.hdf',
    '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4D_tail.rfr001/moments_stats.hdf',
]

In [ ]:
moments = ['n']
for file_name in files:
    hdf = h5py.File(file_name, 'r')
    sizes = hdf['sizes'][:]
    r2 = {m: hdf[m]['r2'][:] for m in moments}

    print(os.path.basename(file_name), r2['n'].max())


In [ ]:
moments = ['vx', 'vy', 'vz']

fig, axes = plt.subplots(1, 3, sharex='all', sharey='all', figsize=(22, 6), dpi=300)

for file_name in files:
    hdf = h5py.File(file_name, 'r')
    sizes = hdf['sizes'][:]
    r2 = {m: hdf[m]['r2'][:] for m in moments}
    points_true = {m: hdf[m]['points_true'][:] for m in moments}
    points_recon = {m: hdf[m]['points_recon'][:] for m in moments}
    hdf.close()
    
    region_title = 'Phase ' + os.path.basename(os.path.dirname(file_name)).replace("_", " ").split(".")[0]
    region_title = ' '.join([word[0].upper() + word[1:] for word in region_title.split(' ')])

    for i, m in enumerate(r2):
        axes[i].plot(sizes/(32*16*2), r2[m], 'o-', label=region_title)
        axes[i].set_title(m.capitalize(), fontsize=16)
        axes[i].set_ylabel('Correlation Coefficient ($r^2$)', fontsize=16)
        axes[i].set_xlabel('Dimensionality Reduction (Fraction)', fontsize=16)

    axes[i].set_ylim(0.75, 1)
for ax in axes:
    #ax.axhline(1, color='black', linestyle='dashed')
    ax.tick_params(axis='both', which='major', labelsize=16)

axes[-1].legend(ncol=1, bbox_to_anchor = (1.55, .85), loc='center right', fontsize=14)
    
    
fig.suptitle('Neural Network Bulk Velocity Reconstruction Correlation For Each Orbital Configuration', fontweight='bold', fontsize=20)
fig.tight_layout()
fig.savefig('Figure2.jpg')
None

# Figure 3

In [ ]:
def get_data(cdf_filename):

    N_EN = 32
    N_EN_SHELLS = 2
    N_PHI = 32
    N_THETA = 16
    cdf = pycdf.CDF(cdf_filename)

    dist = cdf['mms1_dis_dist_brst'][:]
    dist_err = cdf['mms1_dis_disterr_brst'][:]
    epoch = cdf['Epoch'][:]
    ntime = epoch.size
    counts = np.zeros((ntime, N_PHI, N_THETA, N_PHI))

    for i in range(ntime):
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            tmp_counts = np.square(dist[i] / dist_err[i])
        tmp_counts[np.isnan(tmp_counts)] = 0
        tmp_counts = np.rint(tmp_counts)
        counts[i] = tmp_counts

    cdf.close()

    return epoch, counts, ntime

In [ ]:
test_data = nnet_evaluate.load_test_data('4B_dayside')
f1ct = utils.get_f1ct({'4B_dayside': test_data}, ['4B_dayside'])

In [ ]:
E = np.array([2.160000e+00, 3.910000e+00, 7.070000e+00, 1.093000e+01,
       1.424000e+01, 1.854000e+01, 2.414000e+01, 3.144000e+01,
       4.094000e+01, 5.332000e+01, 6.944000e+01, 9.043000e+01,
       1.177700e+02, 1.533600e+02, 1.997200e+02, 2.601000e+02,
       3.387200e+02, 4.411100e+02, 5.744500e+02, 7.481000e+02,
       9.742300e+02, 1.268720e+03, 1.652240e+03, 2.151680e+03,
       2.802100e+03, 3.649120e+03, 4.752190e+03, 6.188690e+03,
       8.059430e+03, 1.049565e+04, 1.366831e+04, 1.780000e+04],
      dtype=np.float32)

In [ ]:
N_EN = 32
N_EN_SHELLS = 2
N_PHI = 32
N_THETA = 16

#hdf_filename = glob.glob('/home/ubuntu/data/recons/4B_dayside-100/*.hdf5')[95]
hdf_filename = '../compression-cfha_code/example.h5'
print(hdf_filename)
#cdf_filename = '/mnt/efs/dasilva/compression-cfha/data/mms_data/4B_dayside/' + os.path.basename(hdf_filename).replace('.hdf5', '.cdf')
cdf_filename = '../compression-cfha_code/mms1_example.cdf'
gpc_filename = hdf_filename.replace('.h5', '.gpc')
gpc_filename = gpc_filename.replace('.hdf5', '.gpc')

print(gpc_filename)
epoch, counts, ntime = get_data(cdf_filename)

hdf = h5py.File(hdf_filename, 'r')
counts_recon = hdf['counts'][:]
hdf.close()

moms_true = [moms_fast.fast_moments(f1ct * c) for c in counts]
moms_recon = [moms_fast.fast_moments(f1ct * c) for c in counts_recon]


cmpr_ratio =  (32 * 16 * 32 * 16 * epoch.size) / (os.path.getsize(gpc_filename) * 8)
file_size = os.path.getsize(gpc_filename) / epoch.size
vars = ['Strue', 'Srecon', 'Srel']
fig, axes = plt.subplots(len(vars), 1, figsize=(15, 4*len(vars)), dpi=300, sharex=True)


Escale = np.zeros((32, counts.shape[0]))
for k in range(Escale.shape[1]):
    Escale[:, k] = E

caxes = []


a = (counts).mean(axis=((1, 2))).T
b = (counts_recon).mean(axis=((1, 2))).T
rel_error = 100 * (a - b)/a
    
for i, var in enumerate(vars):
    vmin = .01
    vmax = 15
    if var == 'Strue':
        im = axes[i].pcolor(epoch, E, (counts).mean(axis=((1, 2))).T, norm=LogNorm(vmin=vmin, vmax=vmax), cmap='plasma')
        axes[i].set_ylabel('$\\bf{Original~Ion~Data}$\nEnergy [eV]', fontsize=18)
        
        divider = make_axes_locatable(axes[i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical').set_label('Average Counts\nper Energy Channel', fontsize=16)
    elif var == 'Srecon':
        im = axes[i].pcolor(epoch, E, (counts_recon).mean(axis=((1, 2))).T, norm=LogNorm(vmin=vmin, vmax=vmax), cmap='plasma')
        axes[i].set_ylabel('$\\bf{Compressed~Ion~Data}$\nEnergy [eV]', fontsize=18)
        
        divider = make_axes_locatable(axes[i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical').set_label('Average Counts\nper Energy Channel', fontsize=16)
    elif var == 'Srel':
        im = axes[i].pcolor(epoch, E, rel_error, vmin=-.25, vmax=.25, cmap='bwr')
        axes[i].set_ylabel('$\\bf{Relative~Error}$\nEnergy [eV]', fontsize=18)
        #axes[i].set_yscale('log')
        divider = make_axes_locatable(axes[i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        fig.colorbar(im, cax=cax, orientation='vertical').set_label('%', fontsize=16)

    axes[i].set_yscale('log')
    caxes.append(cax)

axes[0].set_title(f'Demonstration of Compression Spectrogram (Compression Ratio: {cmpr_ratio:.1f}X)', fontweight='bold', fontsize=20)
fig.tight_layout()

for ax in axes.tolist() + caxes:
    ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig('Figure3.jpg')

# Figure 4

In [ ]:
vars = ['n', 'vx', 'vy', 'vz', 'txx', 'tyy', 'tzz']
fig, axes = plt.subplots(len(vars), 1, figsize=(15, 2.5*len(vars)), dpi=300, sharex=True)

for i, var in enumerate(vars):

    #axes[i].set_title(var, fontsize=16)
    axes[i].plot(epoch, [d[var] for d in moms_true], label=f'True')
    axes[i].plot(epoch, [d[var] for d in moms_recon], label=f'Reconstructed')
        
    if i == 0:
        axes[i].legend(ncol=2, fontsize=18)
    if var == 'n':
        axes[i].set_ylim([0, 1.1 * np.max([d[var] for d in moms_true])])
        axes[i].set_ylabel('n ($cm^{-3}$)', fontsize=16)
    elif var[0] == 'v':
        axes[i].set_ylabel(f'{var.capitalize()} (km/s)', fontsize=16)
    elif var[0] == 't':
        axes[i].set_ylabel(f'{var.capitalize()} (eV)', fontsize=16)
        axes[i].set_ylim(0, 600)
    axes[i].set_xlim(epoch[0], epoch[-1])

    
axes[0].set_title(f'Demonstration of Compression Moments (Compression Ratio: {cmpr_ratio:.1f}X)', fontweight='bold', fontsize=20)
fig.tight_layout()

for ax in axes.tolist():
    ax.tick_params(axis='both', which='major', labelsize=16)

axes[0].text(epoch[1420], 5, 'Density\nOscillations', fontsize=16)
axes[2].text(epoch[900], -130, 'Vy Direction\nSwitch', fontsize=16)

#axes[1].text(epoch[220], -350, 'Transition\nRegion', fontsize=16)
axes[4].text(epoch[220], 50, 'Transition\nRegion', fontsize=16)
axes[5].text(epoch[220], 50, 'Transition\nRegion', fontsize=16)
axes[6].text(epoch[220], 50, 'Transition\nRegion', fontsize=16)

axes[4].text(epoch[1400], 200, 'Transition\nRegion', fontsize=16)
axes[5].text(epoch[1400], 200, 'Transition\nRegion', fontsize=16)
axes[6].text(epoch[1400], 200, 'Transition\nRegion', fontsize=16)

    
#os.makedirs('plots', exist_ok=True)
fig.subplots_adjust(hspace=.1)
fig.savefig('Figure4.jpg')

# Figure 5

In [ ]:
import h5py, pandas as pd

def get_row(phase, file_name):
    hdf = h5py.File(file_name, 'r')
    i = np.argmin(np.abs(hdf['sizes'][:] - 100))


    row = {'phase': phase}

    for var in ['n', 'vx', 'vy', 'vz', 'txx', 'tyy', 'tzz']:
        points_recon = hdf[var]['points_recon'][i, :]
        points_true = hdf[var]['points_true'][i, :]
        err = (points_recon - points_true) 
        
        ls = sorted(err)
        
        alpha = 0.05
        low = ls[int(alpha/2 * len(ls))]
        hi = ls[int((1-alpha/2) * len(ls))]
        
        row[f'{var}_mean'] = np.mean(err)
        row[f'{var}_std'] = np.std(err)
        row[f'{var}_median'] = np.median(err)
        row[f'{var}_ci_low'] = low
        row[f'{var}_ci_hi'] = hi
        
    return row

    
df_rows = [
    get_row('4A Dusk Flank', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4A_dusk_flank.rfr001/moments_stats.hdf'),
    get_row('4B Dayside', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4B_dayside.rfr001/moments_stats.hdf'),
    get_row('4C Dawn Flank', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4C_dawn_flank.rfr001/moments_stats.hdf'),
    get_row('4D Tail', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4D_tail.rfr001/moments_stats.hdf')
]

df = pd.DataFrame(df_rows)
df

In [ ]:
vars = ['_', 'n', '_', 'vx', 'vy', 'vz', 'txx', 'tyy', 'tzz']

fig, axes = plt.subplots(3, len(vars)//3, sharex='all', figsize=(18, 8), dpi=300)
axes_orig = axes
axes = axes.flatten()

for i, var in enumerate(vars):
    
    if var == '_':
        axes[i].set_axis_off()
        continue
        
    color = {'n': 'blue', 'vx': 'gold', 'vy': 'gold', 'vz': 'gold', 'txx': 'red', 'tyy': 'red', 'tzz': 'red'}[var]
        
    axes[i].errorbar(x=df['phase'], y=df[f'{var}_median'], yerr=(df[f'{var}_median'] - df[f'{var}_ci_low'], df[f'{var}_ci_hi'] - df[f'{var}_median']), color=color, ecolor='k', capsize=3)
    axes[i].plot(df['phase'], df[f'{var}_median'], 'ko')
    if var == 'n':
        axes[i].set_ylabel('n Error ($cm^{-3}$)', fontsize=16)
        #axes[i].set_ylim([-12, 12])
    elif var[0] == 'v':
        axes[i].set_ylabel(f'{var.capitalize()} Error (km/s)', fontsize=16)
        #axes[i].set_ylim([-160, 160])
    elif var[0] == 't':
        axes[i].set_ylabel(f'{var.capitalize()} Error (eV)', fontsize=16)
        #axes[i].set_ylim(-500, 500)
for ax in axes.tolist():
    ax.tick_params(axis='both', which='major', labelsize=16)
    ax.set_xticklabels([s.split(' ', 1)[0] + '\n' + s.split(' ', 1)[1] for s in df.phase])

fig.suptitle(f'95% Confidence Intervals of Signed Moments Error (Compression Ratio: {cmpr_ratio:.1f}X)', fontweight='bold', fontsize=20)
fig.tight_layout()
fig.savefig('Figure5.jpg')

# Table 2

In [ ]:
import h5py, pandas as pd

def get_row(phase, file_name):
    hdf = h5py.File(file_name, 'r')
    i = np.argmin(np.abs(hdf['sizes'][:] - 100))

    row = {'phase': phase}

    for var in ['n', 'vx', 'vy', 'vz', 'txx', 'tyy', 'tzz']:
        points_true = hdf[var]['points_true'][i, :]
                
        ls = sorted(points_true)
        
        alpha = 0.05
        low = ls[int(alpha/2 * len(ls))]
        hi = ls[int((1-alpha/2) * len(ls))]

        row[var] = '%.3f - %.3f' % (low, hi)
    return row

    
df_rows = [
    get_row('4A Dusk Flank', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4A_dusk_flank.rfr001/moments_stats.hdf'),
    get_row('4B Dayside', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4B_dayside.rfr001/moments_stats.hdf'),
    get_row('4C Dawn Flank', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4C_dawn_flank.rfr001/moments_stats.hdf'),
    get_row('4D Tail', '/mnt/efs/dasilva/compression-cfha/data/nnet_models/hidden_layer_exp/4D_tail.rfr001/moments_stats.hdf')
]

pd.DataFrame(df_rows)